In [1]:
from qcodes.instrument_drivers.tektronix.Keithley_s46 import S46, LockAcquisitionError

In [2]:
s46 = S46("s2", "GPIB0::7::INSTR")

Connected to: KEITHLEY INSTRUMENTS INC. SYSTEM 46 (serial:1327388, firmware:A03) in 0.62s


In [3]:
s46.get_closed_channels()

[]

In [4]:
len(s46.channels)

26

In [5]:
s46.A1.state("open")

In [6]:
s46.A1.state()

'open'

In [7]:
s46.A1.state("close")

In [8]:
s46.A1.state()

'close'

In [9]:
try: 
    s46.A2.state("close")
    raise("We should not be here")
except LockAcquisitionError as e: 
    print(e)

('Relay A is already in use by channel 1', 'setting s2_A2_state to close')


In [10]:
s46.A1.state("open")

In [11]:
s46.A2.state("close")

In [12]:
s46.A2.state("open")

In [13]:
assert s46.A1 is s46.channels[0]

In [14]:
assert s46.A2 is s46.channels[1]

In [15]:
assert s46.B1 is s46.channels[6]

In [16]:
s46.channels[6].short_name

'B1'

In [29]:
s46.A1.state("close")

In [30]:
s46.B1.state("close")

In [31]:
s46.R8.state("close")

In [32]:
s46.get_closed_channels()

[<S46Channel: s2_A1 of S46: s2>,
 <S46Channel: s2_B1 of S46: s2>,
 <S46Channel: s2_R8 of S46: s2>]

In [33]:
s46.get_closed_channel_numbers()

[1, 7, 32]

In [35]:
print(S46.alias_to_channel_number("A1"))
print(S46.alias_to_channel_number("B1"))
print(S46.alias_to_channel_number("R8"))

1
7
32


In [38]:
def channel_number_to_alias(num): 
    n = num - 24
    if n > 0: 
        return f"R{n}"
    index = num % 6
    relay = {0: "A", 1: "B", 2: "C", 3: "D"}[(num - 1)//6]
    return f"{relay}{index}"

In [40]:
print(channel_number_to_alias(1))
print(channel_number_to_alias(7))
print(channel_number_to_alias(32))

A1
B1
R8


In [41]:
print(channel_number_to_alias(2))
print(channel_number_to_alias(8))
print(channel_number_to_alias(31))

A2
B2
R7


In [26]:
s46.open_all_channels()

In [27]:
s46.get_closed_channels()

[]

In [28]:
s46.A1.state("open")